In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import requests
response = requests.get("https://huggingface.co")
print(response.status_code)  # Should print 200

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

# 1. Verify GPU availability
if not torch.cuda.is_available():
    raise RuntimeError("GPU not available. Please enable GPU in Kaggle settings.")
print(f"GPU Device: {torch.cuda.get_device_name(0)}")

# 2. Load and clean dataset
try:
    df = pd.read_csv("/kaggle/input/bert-scam/SyntheticDataSet_Call_Text_5.csv")  # Adjust path for Kaggle
except FileNotFoundError:
    raise FileNotFoundError("Dataset file 'SyntheticDataSet.txt' not found in /kaggle/input/")

df = df[['text', 'label']].rename(columns={'text': 'text', 'label': 'label'})
df = df.dropna(subset=['text', 'label'])
# Normalize labels and cast to integers
df['label'] = df['label'].str.lower()
df['label'] = df['label'].map({'fraud': 1, 'normal': 0})
df['label'] = df['label'].astype(int)
print(f"Label dtype: {df['label'].dtype}")
print(f"Unique labels: {df['label'].unique()}")
assert df['label'].isnull().sum() == 0, "Label mapping failed."
assert df['label'].isin([0, 1]).all(), "Labels contain unexpected values."
print(f"Label distribution: {df['label'].value_counts()}")
print(f"Text length stats: {df['text'].str.len().describe()}")
print(f"Duplicates: {df.duplicated().sum()}")

# 3. Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))

# 4. Tokenization
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(example):
    return tokenizer(example['text'], truncation=True, padding="max_length", max_length=128)

train_ds = train_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)
train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# 5. Load model
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model.to("cuda")  # Move model to GPU

# 6. Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# 7. Define training args
training_args = TrainingArguments(
    output_dir="/kaggle/working/results",  # Kaggle output directory
    eval_strategy="epoch",
    per_device_train_batch_size=32,  # Increased for GPU
    per_device_eval_batch_size=64,   # Increased for GPU
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="/kaggle/working/logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    logging_strategy="epoch",
    fp16=True  # Enable mixed precision for GPU
)

# 8. Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

# 9. Train
trainer.train()

# 10. Save model
model_path = "/kaggle/working/bert-spam-model2"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

# 11. Optional: Save model to Kaggle output for download
import shutil
shutil.make_archive("/kaggle/working/bert-spam-model2", 'zip', model_path)
print(f"Model saved and zipped at /kaggle/working/bert-spam-model2.zip")

In [ ]:
from IPython.display import FileLink
FileLink(r'bert-spam-model2.zip')